In [1]:
# Importing the necessary libraries
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import sklearn
import scipy as sp
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from kneed import KneeLocator
import sys

In [2]:
# Calculating Cumulative Count Cut
def scaleCCC(x):
  return((x - np.nanpercentile(x, 2))/(np.nanpercentile(x, 98) - np.nanpercentile(x,2)))

def applyCCC(img):
  # Extracting individual bands(4,8,12) from the uploaded images
  band4 = img.GetRasterBand(1).ReadAsArray()
  band8 = img.GetRasterBand(2).ReadAsArray()
  band12 = img.GetRasterBand(3).ReadAsArray()
  ccc4 = scaleCCC(band4)
  ccc8 = scaleCCC(band8)
  ccc12 = scaleCCC(band12)
  # Creating a new image by combining the three bands
  return(np.dstack((ccc4,ccc8,ccc12)))

In [3]:
def process_image(img):
    data = img.ReadAsArray()
    data = np.moveaxis(data, 0, -1)
    rows, cols, bands = data.shape
    data_2d = data.reshape(rows*cols, bands)
    data_2d_scaled = scaler.fit_transform(data_2d)
    data_2d_scaled = np.nan_to_num(data_2d_scaled)
    return data_2d_scaled

In [4]:
# Defining Plot Function

def plot_image(plt_img):
  plt.figure()
  plt.imshow(plt_img)
  plt.show()

In [5]:
# Running the DBSCAN Algorithm
first = True
first2 = True
scaler = StandardScaler()

for i in range(3,71):
    # List of images
    images_list=[str(i)+ "_P_256.tif", str(i) +"_A_256.tif"]
    images = []
    # Opening the satellite images
    for img_name in images_list:
        img = gdal.Open('./ALL_FLOOD_256/'+ img_name)
        data_2d_scaled = process_image(img)

        if(first):
            train_image=data_2d_scaled
            first = False
        else:
            train_image = np.concatenate([data_2d_scaled, train_image], axis=0)
    if (first2):
        combined_data = train_image
    else:
        combined_data = np.concatenate([combined_data, train_image], axis=0)

In [6]:
# Opening the test images
imgP = gdal.Open("./ALL_FLOOD_256/2_P_256.tif")
imgA = gdal.Open("./ALL_FLOOD_256/2_A_256.tif")

before_img = applyCCC(imgP)
after_img = applyCCC(imgA)

data_2d_scaled_before = process_image(imgP)
data_2d_scaled_after = process_image(imgA)

In [7]:
images = []
for i in range(3,71):
    # List of images
    images_list=[str(i)+ "_P_256.tif", str(i) +"_A_256.tif"]
    for img_name in images_list:
        img = gdal.Open('./ALL_FLOOD_256/'+ img_name)
        images.append(img)

In [8]:
import csv
# headerList = ["Min Samples","Epsilon","No. of Before Clusters", "No. of After Clusters", "No. of Before Noise points", "No. of After Noise points"]

# with open('./myfile.csv', 'a', newline='') as f:
#     writer = csv.DictWriter(f,fieldnames=headerList)
#     writer.writeheader()

for k in range(20, 10000, 10):
    nearest_neighbors = NearestNeighbors(n_neighbors=k)
    neighbors = nearest_neighbors.fit(combined_data)
    distances, indices = neighbors.kneighbors(combined_data)
    distances = np.sort(distances[:,k-1], axis=0)
    i = np.arange(len(distances))
    knee = KneeLocator(i, distances, S=1, curve='convex', direction='increasing', interp_method='polynomial')
    epsilon = distances[knee.knee]
    epsilon -= 0.05
    max_epsilon = epsilon+0.1

    while(epsilon < max_epsilon):
        
        dbscan_model = DBSCAN(eps=epsilon, min_samples = k)
        first = True
        first2 = True
        for i in range(0, len(images), 2):
            for j in range(2):
                img = images[i+j]
                data_2d_scaled = process_image(img)
                if(first):
                    train_image=data_2d_scaled
                    first = False
                else:
                    train_image = np.concatenate([data_2d_scaled, train_image], axis=0)
            if (first2):
                combined_data = train_image
            else:
                combined_data = np.concatenate([combined_data, train_image], axis=0)
            dbscan_model.fit(combined_data)
            print(dbscan_model.labels_.shape)
        
        dbscan_before_labels = dbscan_model.fit_predict(data_2d_scaled_before)
        core_samples_mask_before = np.zeros_like(dbscan_before_labels, dtype=bool)
        core_samples_mask_before[dbscan_model.core_sample_indices_] = True
        labels_before = dbscan_before_labels
        n_clusters_before = len(set(labels_before))
        n_noise_before = list(labels_before).count(-1)

        dbscan_after_labels = dbscan_model.fit_predict(data_2d_scaled_after)
        core_samples_mask_after = np.zeros_like(dbscan_after_labels, dtype=bool)
        core_samples_mask_after[dbscan_model.core_sample_indices_] = True
        labels_after = dbscan_after_labels
        n_clusters_after = len(set(labels_after))
        n_noise_after = list(labels_after).count(-1)

        silhouetteScore = sklearn.metrics.silhouette_score(data_2d_scaled_after, dbscan_after_labels)
        accuracy = ((silhouetteScore+1)/2)*100

        with open('tuning.csv', 'a', newline='') as f:
            writer = csv.writer(f)
            print([k, epsilon, n_clusters_before, n_noise_before, n_clusters_after, n_noise_after, silhouetteScore, accuracy])
            writer.writerow([k, epsilon, n_clusters_before, n_noise_before, n_clusters_after, n_noise_after, silhouetteScore, accuracy])

In [ ]:
sys.argv = [{"test_var":"Testing","successful_execution":True}]
execfile("mailing.py")